<a href="https://colab.research.google.com/github/ash-rulz/RAG/blob/main/RAG_Example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below excercise is from a [post](https://www.linkedin.com/pulse/get-insight-from-your-business-data-build-llm-application-jain/) by Ashish Jain. Full credits goes to him.

We are using FLAN_T5 model in this excercise. The predecessor of this model is the T5 model which originated from the paper - Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer(see [video](https://www.youtube.com/watch?v=91iLu6OOrwk) for more details on the paper, or even detailed video [here](https://www.youtube.com/watch?v=Axo0EtMUK90)). The T5 model is based on the paper Scaling Instruction-Finetuned Language Models(see [video](https://www.youtube.com/watch?v=SHMsdAPo2Ls)). FLAN T5 is just Fine-tuned LANguage model on T5.

Next, we use [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) which converts the sentences to a 384 dimensional vector space.

In [3]:
!pip install transformers

from transformers import AutoModel, AutoTokenizer

# Download the MiniLM model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Download the flan-t5-large model
model_name_flan = "google/flan-t5-large"
model_flan = AutoModel.from_pretrained(model_name_flan)
tokenizer_flan = AutoTokenizer.from_pretrained(model_name_flan)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
!pip install langchain
!pip install pypdf

In [6]:
#Load the pdf to memory
from langchain.document_loaders import PyPDFLoader
pdfLoader = PyPDFLoader("example_data/LORA.pdf")
documents = pdfLoader.load()

In [10]:
len(documents)

26

In [9]:
#Split the file to chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

In [14]:
len(docs)

104

In [15]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=57703eb0a9c36c936881753e2b53c0c9f8c397c6102a7bfa6f21de91b6781593
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

model_path = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [27]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.0 MB/s eta 0:00:00


In [28]:
#Create vector store
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [32]:
question = "What are the hyperparameters in LORA?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

rq=rv= 32,lp= 8,li= 4 151.1 M 75.9 91.1
rq=rv= 64,lp= 8,li= 4 302.1 M 76.2 91.3
LoRA+PL rq=rv= 8,lp= 8,li= 4 52.8 M 72.9 90.2
Table 15: Hyperparameter analysis of different adaptation approaches on WikiSQL and MNLI. Both
preﬁx-embedding tuning (PreﬁxEmbed) and preﬁx-layer tuning (PreﬁxLayer) perform worse as we
increase the number of trainable parameters, while LoRA’s performance stabilizes. Performance is
measured in validation accuracy.
Method MNLI(m)-100 MNLI(m)-1k MNLI(m)-10k MNLI(m)-392K
GPT-3 (Fine-Tune) 60.2 85.8 88.9 89.5
GPT-3 (PreﬁxEmbed) 37.6 75.2 79.5 88.6
GPT-3 (PreﬁxLayer) 48.3 82.5 85.9 89.6
GPT-3 (LoRA) 63.8 85.6 89.2 91.7
Table 16: Validation accuracy of different methods on subsets of MNLI using GPT-3 175B. MNLI-
ndescribes a subset with ntraining examples. We evaluate with the full validation set. LoRA
performs exhibits favorable sample-efﬁciency compared to other methods, including ﬁne-tuning.
To be concrete, let the singular values of Ui⊤
AUj
